### 題目 : 今天要不要出門
### 屬性 :
- 昨天有出門 : wasOut
- 昨天有下雨 : wasRainy
- 今天有下雨 : isRainy
- 今天是假日 : isHoliday
- 今天想睡覺 : isSleepy
- 功課有寫完 : doneHomework
- 家事有做完 : doneHoueswork

### 規則 : 

![tree](tree.jpg)

### 資料 :

- 亂數產生**1000**筆資料
     
         

In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
def getData(numInstance=1000):
    numInstance = 1000
    df = pd.DataFrame(columns=['wasOut', 'wasRainy', 'isRainy', 'isHoliday', 'isSleepy', 'doneHomework', 'doneHousework', 'out'])
    for col in df.columns[:-1]:
        df[col]=[bool(random.randint(0,1)) for i in range(numInstance)]
    for i in range(numInstance):
        if df.loc[i, 'wasRainy']:
            if df.loc[i, 'isRainy']:
                df.loc[i, 'out'] = False
            else:
                if df.loc[i, 'doneHousework']:
                    if df.loc[i, 'wasOut']:
                        df.loc[i, 'out'] = False
                    else:
                        df.loc[i, 'out'] = True
                else:
                    df.loc[i, 'out'] = False
        else:
            if df.loc[i, 'doneHomework']:
                if df.loc[i, 'isHoliday']:
                    df.loc[i, 'out'] = True
                else:
                    if df.loc[i, 'isSleepy']:
                        df.loc[i, 'out'] = False
                    else:
                        df.loc[i, 'out'] = True
            else:
                df.loc[i, 'out'] = False
    return df

In [3]:
# Generate new data
# df = getData(1000)
# df.to_csv('Data.csv', index=False)

# Or read data form file
df = pd.read_csv('data.csv')

In [4]:
count_Nodes = np.zeros(8)
for i in range(df.shape[0]):
        if df.loc[i, 'wasRainy']:
            if df.loc[i, 'isRainy']:
                count_Nodes[0] +=1
            else:
                if df.loc[i, 'doneHousework']:
                    if df.loc[i, 'wasOut']:
                        count_Nodes[1] += 1
                    else:
                        count_Nodes[2] += 1
                else:
                    count_Nodes[3] += 1
        else:
            if df.loc[i, 'doneHomework']:
                if df.loc[i, 'isHoliday']:
                    count_Nodes[4] += 1
                else:
                    if df.loc[i, 'isSleepy']:
                        count_Nodes[5] += 1
                    else:
                        count_Nodes[6] += 1
            else:
                count_Nodes[7] += 1

In [5]:
class treeNode():
    def __init__(self, value, rightSon=None, leftSon=None):
        self.value = value
        self.rightSon = rightSon
        self.leftSon = leftSon
    def getGINI_son(self, df):
        counts = df['out'].value_counts()
        total = sum(counts)
        GINI = 1 - sum([pow(count/total, 2) for count in counts])
        return total, GINI
        # return GINI
    def getGINI(self, df):
        N1_count, N1_GINI = self.getGINI_son(df[df[self.value]==True])
        N2_count, N2_GINI = self.getGINI_son(df[df[self.value]==False])
        count = N1_count + N2_count
        GINI = (N1_count*N1_GINI + N2_count*N2_GINI) / count
        # print(N1_count, N1_GINI, N2_count, N2_GINI)
        return GINI
    def getValue(self):
        return self.value
    def getRightSon(self):
        return self.rightSon
    def getLeftSon(self):
        return self.leftSon

In [6]:
#
Node_wasOut = treeNode('wasOut')
Node_doneHousework = treeNode('doneHousework', rightSon=Node_wasOut)
Node_isRainy = treeNode('isRainy', leftSon=Node_doneHousework)
#
Node_isSleepy = treeNode('isSleepy')
Node_isHoliday = treeNode('isHoliday', leftSon=Node_isSleepy)
Node_doneHomework = treeNode('doneHomework', rightSon=Node_isHoliday)
#
Node_wasRainy = treeNode('wasRainy', Node_isRainy, Node_doneHomework)


In [7]:
def printGINI(Node, df):
    if Node.getRightSon() != None:
        printGINI(Node.getRightSon(), df[df[Node.getValue()]==True])
    if Node.getLeftSon() != None:
        printGINI(Node.getLeftSon(), df[df[Node.getValue()]==False])
    print(Node.getValue(), round(Node.getGINI(df), 3))

In [8]:
printGINI(Node_wasRainy,df)

wasOut 0.0
doneHousework 0.243
isRainy 0.188
isSleepy 0.0
isHoliday 0.247
doneHomework 0.196
wasRainy 0.343


In [40]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(df.iloc[:, :-1], df['out'])

In [44]:
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=df.columns[:-1],  
                                class_names=['in', 'Out'],
                                filled=True, rounded=True,
                                special_characters=True)  
graph = graphviz.Source(dot_data)  
graph.render('tree')

'tree.pdf'

In [47]:
for col in df.columns[:-1]:
    # right
    N1_count = df[df[col]==True].shape[0]
    N2_count = df[df[col]==False].shape[0]
    N1_C1, N1_C2 = df[df[col]==True]['out'].value_counts()
    N2_C1, N2_C2 = df[df[col]==False]['out'].value_counts()
    GINI = (N1_count * (1 - pow(N1_C1 / (N1_C1 + N1_C2), 2) - pow(N1_C2 / (N1_C1 + N1_C2), 2)) +
            N2_count * (1 - pow(N2_C1 / (N2_C1 + N2_C2), 2) - pow(N2_C2 / (N2_C1 + N2_C2), 2)))/(N1_count + N2_count)
    print(col, round(GINI, 2))
        

wasOut 0.36
wasRainy 0.34
isRainy 0.37
isHoliday 0.37
isSleepy 0.37
doneHomework 0.31
doneHousework 0.36
